In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
matchups = pd.read_csv("data\\matchups\\model_matchups.csv", index_col=0)

In [ ]:
y = matchups['outcome']
x_train = matchups.drop(columns=['outcome','game_code','batter_game_code','pitcher_game_code','batter_id','pitcher_id','b_pos'])

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
labels_enc = le.transform(y)
labels = tf.keras.utils.to_categorical(labels_enc)

ss = preprocessing.StandardScaler()
x_standardized = ss.fit_transform(x_train)

In [ ]:
p = .1
inputs = tf.keras.layers.Input((74,), name='numeric_inputs')

x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(250, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(50, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(55, activation='softmax', name='output')(x)

In [ ]:
model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# model.summary()

In [ ]:
def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(x_standardized.shape[0], batch_size)
        yield ({'numeric_inputs': x_standardized[batch_idx]},
               {'output': labels[batch_idx]})

batch_size = 128

model.fit(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=5,
    max_queue_size=10  
)
